# OCR（文字認識）

このノートブックでは、ocr（文字認識）について学びます。

## 目次
1. [Tesseract OCR](#tesseract-ocr)
2. [前処理](#前処理)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import filters, feature

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False
np.random.seed(42)
print("ライブラリのインポートが完了しました。")

## Tesseract OCR

In [ ]:
# Tesseract OCRによる文字認識
# 注意: pytesseractをインストールする必要があります
# pip install pytesseract

try:
    import pytesseract
    TESSERACT_AVAILABLE = True
except ImportError:
    TESSERACT_AVAILABLE = False
    print("警告: pytesseractがインストールされていません。")
    print("インストール: pip install pytesseract")
    print("また、Tesseract OCR本体もインストールする必要があります。")

# サンプルテキスト画像の作成
img_text = np.ones((100, 400, 3), dtype=np.uint8) * 255
cv2.putText(img_text, 'Hello World', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
cv2.putText(img_text, '12345', (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

# グレースケールに変換
gray_text = cv2.cvtColor(img_text, cv2.COLOR_RGB2GRAY)

# OCRの実行（Tesseractが利用可能な場合）
if TESSERACT_AVAILABLE:
    try:
        # テキストの抽出
        text = pytesseract.image_to_string(gray_text)
        
        # 詳細情報の取得
        data = pytesseract.image_to_data(gray_text, output_type=pytesseract.Output.DICT)
        
        print("=== OCR結果 ===")
        print(f"認識されたテキスト: {text}")
        print(f"\n詳細情報:")
        print(f"検出された単語数: {len([w for w in data['text'] if w.strip()])}")
    except Exception as e:
        print(f"OCRエラー: {e}")
        print("Tesseract OCRが正しくインストール・設定されていない可能性があります。")
else:
    # Tesseractが利用できない場合のデモ
    print("=== OCRデモ（Tesseract未インストール） ===")
    print("実際の使用には以下が必要です:")
    print("1. Tesseract OCRのインストール")
    print("2. pytesseractパッケージのインストール: pip install pytesseract")

# 画像の表示
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(img_text)
axes[0].set_title('元のテキスト画像')
axes[0].axis('off')

axes[1].imshow(gray_text, cmap='gray')
axes[1].set_title('グレースケール画像')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\nOCR前処理の重要性:")
print("- 画像の品質向上（ノイズ除去、コントラスト調整）")
print("- 二値化（閾値処理）")
print("- スケーリング（文字サイズの調整）")
print("- 傾き補正")

## 前処理

In [ ]:
# OCR前処理パイプライン
# 文字認識の精度を向上させるための前処理

# ノイズ付きテキスト画像の作成
img_noisy = np.ones((100, 400, 3), dtype=np.uint8) * 255
cv2.putText(img_noisy, 'Hello World', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
# ノイズを追加
noise = np.random.randint(0, 50, (100, 400, 3), dtype=np.uint8)
img_noisy = np.clip(img_noisy.astype(np.int16) - noise, 0, 255).astype(np.uint8)

# グレースケール変換
gray = cv2.cvtColor(img_noisy, cv2.COLOR_RGB2GRAY)

# 前処理1: ガウシアンフィルタでノイズ除去
denoised = cv2.GaussianBlur(gray, (3, 3), 0)

# 前処理2: コントラスト調整（CLAHE）
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced = clahe.apply(denoised)

# 前処理3: 二値化（Otsu法）
_, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 前処理4: モルフォロジー演算でノイズ除去
kernel = np.ones((2, 2), np.uint8)
morph = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

# 前処理5: リサイズ（解像度の調整）
resized = cv2.resize(morph, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img_noisy)
axes[0, 0].set_title('元の画像（ノイズ付き）')
axes[0, 0].axis('off')

axes[0, 1].imshow(gray, cmap='gray')
axes[0, 1].set_title('グレースケール')
axes[0, 1].axis('off')

axes[0, 2].imshow(denoised, cmap='gray')
axes[0, 2].set_title('ノイズ除去')
axes[0, 2].axis('off')

axes[1, 0].imshow(enhanced, cmap='gray')
axes[1, 0].set_title('コントラスト調整')
axes[1, 0].axis('off')

axes[1, 1].imshow(binary, cmap='gray')
axes[1, 1].set_title('二値化（Otsu）')
axes[1, 1].axis('off')

axes[1, 2].imshow(resized, cmap='gray')
axes[1, 2].set_title('リサイズ（2倍）')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("OCR前処理のステップ:")
print("1. ノイズ除去: ガウシアンフィルタ、メディアンフィルタ")
print("2. コントラスト調整: CLAHE、ヒストグラム均等化")
print("3. 二値化: Otsu法、適応的閾値処理")
print("4. モルフォロジー演算: ノイズ除去、文字の補完")
print("5. リサイズ: 適切な解像度への調整")
print("6. 傾き補正: 必要に応じて回転")